# CAI Lab Session 4: Implementing search in the vector space model

In this session you will:

- Continue to work with the `arxiv` repository from last session
- Learn how to do atomic, conjunctive and disjunctive search with ElasticSearch
- Build an inverted index for the `arxiv` repository from last session (should fit in main memory)
- Implement search in the vector space model and compare it with ElasticSearch built-in search mechanism
- Compare different implementations of search

## 1. Built-in search in ElasticSearch

ElasticSearch provides a search mechanism to make queries against a database. 
In the next code snippet you can find examples on how to do this with an atomic query (single term)
and with conjunctive and disjunctive queries.

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl.query import Q


client = Elasticsearch("http://localhost:9200", request_timeout=1000)
s = Search(using=client, index='arxiv')

## atomic query
q = Q('query_string',query='computer')  # Feel free to change the word

s = s.query(q)
response = s[:5].execute()
for r in response:  # only returns a specific number of results
    print('ID= %s SCORE=%s' % (r.meta.id,  r.meta.score))
    print('PATH= %s' % r.path)
    print('TEXT: %s' % r.text[:90])
    print()

ID= b-dDGosBAcLkfcUZVIXu SCORE=6.597525
PATH= /tmp/arxiv/cs.updates.on.arXiv.org/009984
TEXT: In computational science and in computer science, research software is a central asset for

ID= ledDGosBAcLkfcUZWpl5 SCORE=6.536022
PATH= /tmp/arxiv/cs.updates.on.arXiv.org/014543
TEXT: Efficient rendering of photo-realistic virtual worlds is a long standing effort of compute

ID= ludDGosBAcLkfcUZT3JP SCORE=6.3829436
PATH= /tmp/arxiv/quant-ph.updates.on.arXiv.org/001488
TEXT: The design of new devices and experiments in science and engineering has historically reli

ID= w-dDGosBAcLkfcUZXKB1 SCORE=6.3829436
PATH= /tmp/arxiv/cs.updates.on.arXiv.org/013539
TEXT: The design of new devices and experiments in science and engineering has historically reli

ID= mudDGosBAcLkfcUZU4C8 SCORE=6.3366957
PATH= /tmp/arxiv/cs.updates.on.arXiv.org/012998
TEXT: The commercialization of transistors capable of both switching and amplification in 1960 r



In [2]:
## conjunctive query

client = Elasticsearch("http://localhost:9200", request_timeout=1000)
s = Search(using=client, index='arxiv')

q = Q('query_string',query='computer') & Q('query_string',query='magic')

s = s.query(q)
response = s[0:5].execute()
for r in response:  # only returns a specific number of results
    print(f'ID= {r.meta.id} SCORE={r.meta.score}')
    print(f'PATH= {r.path}')
    print(f'TEXT: {r.text[:90]}')
    print()

ID= 1edDGosBAcLkfcUZTnGi SCORE=15.135108
PATH= /tmp/arxiv/quant-ph.updates.on.arXiv.org/000677
TEXT: We give a new algorithm for computing the robustness of magic - a measure of the utility o

ID= c-dDGosBAcLkfcUZT3RR SCORE=15.135108
PATH= /tmp/arxiv/quant-ph.updates.on.arXiv.org/000650
TEXT: We give a new algorithm for computing the robustness of magic - a measure of the utility o

ID= wOdDGosBAcLkfcUZTW_k SCORE=12.778042
PATH= /tmp/arxiv/quant-ph.updates.on.arXiv.org/001652
TEXT: A defining feature in the field of quantum computing is the potential of a quantum device 

ID= QudDGosBAcLkfcUZbeou SCORE=11.47281
PATH= /tmp/arxiv/cond-mat.updates.on.arXiv.org/000521
TEXT: Smale's 7-th problem concerns N-point configurations on the 2-dim sphere which minimize th

ID= TehDGosBAcLkfcUZgUnR SCORE=11.47281
PATH= /tmp/arxiv/math.updates.on.arXiv.org/000731
TEXT: Smale's 7-th problem concerns N-point configurations on the 2-dim sphere which minimize th



In [3]:
## disjunctive query

client = Elasticsearch("http://localhost:9200", request_timeout=1000)
s = Search(using=client, index='arxiv')

q = Q('query_string',query='computer') | Q('query_string',query='magic')

s = s.query(q)
response = s[0:5].execute()
for r in response:  # only returns a specific number of results
    print(f'ID= {r.meta.id} SCORE={r.meta.score}')
    print(f'PATH= {r.path}')
    print(f'TEXT: {r.text[:90]}')
    print()

ID= 1edDGosBAcLkfcUZTnGi SCORE=15.135108
PATH= /tmp/arxiv/quant-ph.updates.on.arXiv.org/000677
TEXT: We give a new algorithm for computing the robustness of magic - a measure of the utility o

ID= c-dDGosBAcLkfcUZT3RR SCORE=15.135108
PATH= /tmp/arxiv/quant-ph.updates.on.arXiv.org/000650
TEXT: We give a new algorithm for computing the robustness of magic - a measure of the utility o

ID= wOdDGosBAcLkfcUZTW_k SCORE=12.778042
PATH= /tmp/arxiv/quant-ph.updates.on.arXiv.org/001652
TEXT: A defining feature in the field of quantum computing is the potential of a quantum device 

ID= Z-dDGosBAcLkfcUZa-J7 SCORE=11.627468
PATH= /tmp/arxiv/cond-mat.updates.on.arXiv.org/003482
TEXT: When two monolayers of graphene are stacked with a small relative twist angle, the resulti

ID= QudDGosBAcLkfcUZbeou SCORE=11.47281
PATH= /tmp/arxiv/cond-mat.updates.on.arXiv.org/000521
TEXT: Smale's 7-th problem concerns N-point configurations on the 2-dim sphere which minimize th



## 2. Excruciatingly slow search

In class we have presented a _slow_ version of search that, given a search query $q$, loops over every document in the database
computing the cosine similarity between document and query. Once this is done, it sorts documents by their similarity w.r.t. $q$ and returns the top $r$
scoring ones. 

```
1. for each d in D:
    sim(d,q) = 0
    get vector representing d
    for each w in q:
        sim(d,q) += tf(d,w) * idf(w)
    normalize sim(d,q) by |d|*|q|
2. sort results by similarity
3. return top r docs
```

A possible implementation can be found below.

In [3]:
from elasticsearch.helpers import scan
from pprint import pprint
from elasticsearch import Elasticsearch
import tqdm
import numpy as np

client = Elasticsearch("http://localhost:9200", request_timeout=1000)

r = 10  # only return r top docs
query = 'computer magic'
sims = dict()

l2query  = np.sqrt(len(query.split()))  # l2 of query assuming 0-1 vector representation

# get nr. of docs; just for the progress bar
ndocs = int(client.cat.count(index='arxiv', format = "json")[0]['count'])

# scan through docs, compute cosine sim between query and each doc
for s in tqdm.tqdm(scan(client, index='arxiv', query={"query" : {"match_all": {}}}), total=ndocs):
    docid = s['_source']['path']   # use path as id
    weights = dict(normalize(tf_idf(s['_id'])))   # gets weights as a python dict of term -> weight
    sims[docid] = 0.0
    for w in query.split():  # gets terms as a list
        if w in weights:    # probably need to do something fancier to make sure that word is in vocabulary etc.
            sims[docid] += weights[w]   # accumulates if w in current doc
    # normalize sim
    sims[docid] /= l2query

# now sort by cosine similarity
sorted_answer = sorted(sims.items(), key=lambda kv: kv[1], reverse=True)
pprint(sorted_answer[:r])


100%|██████████| 58102/58102 [05:49<00:00, 166.07it/s]

[('/tmp/arxiv/quant-ph.updates.on.arXiv.org/000650', 0.46298539019176793),
 ('/tmp/arxiv/quant-ph.updates.on.arXiv.org/000677', 0.46285572520081464),
 ('/tmp/arxiv/cond-mat.updates.on.arXiv.org/003482', 0.41693456487012037),
 ('/tmp/arxiv/quant-ph.updates.on.arXiv.org/001475', 0.3078298379878905),
 ('/tmp/arxiv/astro-ph.updates.on.arXiv.org/002083', 0.26997407750109564),
 ('/tmp/arxiv/math.updates.on.arXiv.org/002825', 0.2637693594252774),
 ('/tmp/arxiv/astro-ph.updates.on.arXiv.org/001294', 0.2583918756706909),
 ('/tmp/arxiv/hep-th.updates.on.arXiv.org/000265', 0.2554940223200789),
 ('/tmp/arxiv/math.updates.on.arXiv.org/000955', 0.2554940223200789),
 ('/tmp/arxiv/hep-th.updates.on.arXiv.org/000255', 0.2505494496917682)]


In [24]:
nz = len([x for x, s in sorted_answer if s>0])
total = len(sorted_answer)
print(f'There are {nz} docs with non-zero similarity out of {total}, i.e. {100.0*nz/total:.1f}%')

There are 1948 docs with non-zero similarity out of 58102, i.e. 3.4%


## 3. Your tasks

---

**Exercise 1:**  

Make sure you understand the algorithm for implementing search described in the lecture notes. Both slow and efficient versions. Describe
the number of sums you need to do in both slow and quick versions for the following toy example with a vocabulary of size 4 and four documents:

- $q = 0,1,1,0$

- document-term matrix:
<center>


|        | t1  | t2  | t3  | t4  |
|--------|-----|-----|-----|-----|
| **d1** | 1.2 | 0.0 | 0.0 | 0.0 |
| **d2** | 0.7 | 0.3 | 1.5 | 0.1 |
| **d3** | 0.0 | 0.0 | 0.0 | 0.7 |
| **d4** | 2.0 | 0.0 | 0.0 | 0.0 |

</center>

---

**Exercise 2:**

Implement the quick version; run both slow and quick versions and report times (as a reference, in my old laptop it takes around 5m30s to run the slow version in the code above). Make sure both versions return the same answer. Note that you will need to build an inverted index in order to implement the efficient version as explained in class; it may take time but this is done once for all queries, and can be done "off-line".

---

**Exercise 3:**

Compare the results for a few sample queries that you get from your quick version and ElasticSearch search. Do you get similar results? Which is faster?

---

## 4. Rules of delivery

- To be solved in _pairs_.

- No plagiarism; don't discuss your work with other teams. You can ask for help to others for simple things, such as recalling a python instruction or module, but nothing too specific to the session.

- If you feel you are spending much more time than the rest of the classmates, ask us for help. Questions can be asked either in person or by email, and you'll never be penalized by asking questions, no matter how stupid they look in retrospect.

- Write a short report listing the solutions to the exercises proposed. Include things like the important parts of your implementation (data structures used for representing objects, algorithms used, etc). You are welcome to add conclusions and findings that depart from what we asked you to do. We encourage you to discuss the difficulties you find; this lets us give you help and also improve the lab session for future editions.

- Turn the report to PDF. Make sure it has your names, date, and title. Include your code in your submission.

- Submit your work through the [raco](http://www.fib.upc.edu/en/serveis/raco.html) _before November 6th, 2023_.